In [3]:
import numpy as np
node_n = 8

np.linspace(0, 1, node_n+2).tolist()[1:-1]

[0.1111111111111111,
 0.2222222222222222,
 0.3333333333333333,
 0.4444444444444444,
 0.5555555555555556,
 0.6666666666666666,
 0.7777777777777777,
 0.8888888888888888]

In [2]:
# Make topology
topology = Topology(10, "dumbbell", 0.5)
topology.show_adjacency_matrix()
node_n = topology.n
N_OBSERVATIONS = 2
N_ACTIONS = 2

[[0. 1. 1. 1. 1. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 1. 1. 1. 1. 0.]]


In [3]:
import csv

class NetworkEnvironment(gym.Env):
    def __init__(self, topology, n_observations, n_actions, episode_length=300):
        super(NetworkEnvironment, self).__init__()
        self.topology = topology
        self.node_n = topology.n
        self.n_observations = n_observations
        self.n_actions = n_actions
        self.episode_length = episode_length
        self.current_step = 0
        self.aoi = np.ones(self.node_n) / episode_length
        
        self.action_space = spaces.Discrete(n_actions)
        self.observation_space = spaces.Box(low=0, high=1, shape=(self.node_n, n_observations), dtype=np.float32)
        
    def reset(self):
        self.current_step = 0
        self.aoi = np.ones(self.node_n) / self.episode_length
        return self._get_obs()
        
    def _get_obs(self):
        return np.repeat(self.aoi.reshape(-1, 1), self.n_observations, axis=1)
    
    def step(self, actions):
        transmission_probs = np.random.rand(self.node_n)
        successful_transmissions = (transmission_probs < actions).astype(int)
        
        for i in range(self.node_n):
            if successful_transmissions[i] == 1:
                neighbors = np.where(self.topology.adjacency_matrix[i] == 1)[0]
                self.aoi[neighbors] = 1 / self.episode_length
        
        self.aoi += 1 / self.episode_length
        
        self.current_step += 1
        done = self.current_step >= self.episode_length
        
        utility = np.log2(self.aoi + 1).sum()
        reward = utility
        
        return self._get_obs(), reward, done, successful_transmissions
    
    def render(self):
        pass

def calculate_average_utility_and_log(topology, n_observations, n_actions, n_episodes=100, log_file="simulation_logs.csv"):
    env = NetworkEnvironment(topology, n_observations, n_actions)
    total_utility = 0
    log_data = []

    for episode in range(n_episodes):
        obs = env.reset()
        episode_utility = 0
        
        for step in range(env.episode_length):
            actions = np.random.rand(env.node_n)
            obs, reward, done, transmissions = env.step(actions)
            episode_utility += reward

            log_entry = [episode, step] + transmissions.tolist()
            log_data.append(log_entry)
            
            if done:
                break
                
        total_utility += episode_utility
    
    # Save logs to CSV
    with open(log_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        header = ["Episode", "Step"] + [f"Node_{i+1}" for i in range(env.node_n)]
        writer.writerow(header)
        writer.writerows(log_data)
    
    average_utility = total_utility / n_episodes
    return average_utility

# Example usage
topology = Topology(10, "dumbbell", 0.5)
average_utility = calculate_average_utility_and_log(topology, N_OBSERVATIONS, N_ACTIONS, n_episodes=100, log_file="simulation_logs.csv")
print(f"Average Utility over 100 episodes: {average_utility}")


Average Utility over 100 episodes: 40.05595556747705
